# Implementation of Schwarz algorithm

## Improtant Notes
* SchwarzOpt.jl does not yet perform automatic overlap improvement. This means the user needs to provide sufficient overlap such that the optimizer converges.
* Convergence may fail if the user provides non-contiguous subproblems (partitions), which means a subproblem contains distinct sets of unconnected nodes.

In [1]:
using SchwarzOpt, Ipopt
using Gurobi, CPLEX
using Plasmo, PlasmoPlots
using KaHyPar
using JuMP

using MadNLP
using MadNLPHSL

import HSL_jll

In [2]:
function create_dynamic_problem()
    T = 100 # number of time points
    d = sin.(1:T) # disturbance vector

    # create an OptiGraph
    graph = Plasmo.OptiGraph()

    # Add nodes for states and controls
    Plasmo.@optinode(graph, state[1:T])
    Plasmo.@optinode(graph, control[1:T-1])
    
    # Add state variables
    for node in state
        Plasmo.@variable(node, x)
        Plasmo.@constraint(node, x >= 0)
        Plasmo.@objective(node, Min, x^2)
    end
    # Add control variables
    for node in control
        Plasmo.@variable(node, u)
        Plasmo.@constraint(node, u >= -1000)
        Plasmo.@objective(node, Min, u^2)
    end

    # dynamic coupling
    Plasmo.@linkconstraint(graph, [i = 1:T-1], state[i+1][:x] == state[i][:x] + control[i][:u] + d[i])

    # initial condition
    n1 = state[1]
    Plasmo.@constraint(n1, n1[:x] == 0)
    return graph
end

create_dynamic_problem (generic function with 1 method)

In [3]:
graph = create_dynamic_problem()

      OptiGraph: # elements (including subgraphs)
-------------------------------------------------------------------
      OptiNodes:   199            (199)
      OptiEdges:    99             (99)
LinkConstraints:    99             (99)
 sub-OptiGraphs:     0              (0)

In [4]:
# Define function to perform partitioning
function partition_dynamic_problem!(g::OptiGraph)
    hypergraph, projection_map = Plasmo.hyper_graph(g)
    node_vector = KaHyPar.partition(hypergraph, 8, configuration=:edge_cut, imbalance=0.01) # edge_cut
    partition = Plasmo.Partition(node_vector, projection_map)
    Plasmo.apply_partition!(g, partition)
end

partition_dynamic_problem! (generic function with 1 method)

In [5]:
# Perform partitioning
partition_dynamic_problem!(graph)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
+                    _  __     _   _       ____                               + 
+                   | |/ /__ _| | | |_   _|  _ \ __ _ _ __                    + 
+                   | ' // _` | |_| | | | | |_) / _` | '__|                   + 
+                   | . \ (_| |  _  | |_| |  __/ (_| | |                      + 
+                   |_|\_\__,_|_| |_|\__, |_|   \__,_|_|                      + 
+                                    |___/                                    + 
+                 Karlsruhe Hypergraph Partitioning Framework                 + 
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
*******************************************************************************
*                            Partitioning Context                             *
*******************************************************************************
Partitioning Parameters:
  Hype

In [6]:
graph

      OptiGraph: # elements (including subgraphs)
-------------------------------------------------------------------
      OptiNodes:     0            (199)
      OptiEdges:     7             (99)
LinkConstraints:     7             (99)
 sub-OptiGraphs:     8              (8)

In [7]:
#calculate subproblems using expansion distance
subgraphs = Plasmo.getsubgraphs(graph)
expanded_subgraphs = Plasmo.expand.(graph,subgraphs,5)

8-element Vector{OptiGraph}:
       OptiGraph: # elements (including subgraphs)
-------------------------------------------------------------------
      OptiNodes:    43             (43)
      OptiEdges:    21             (21)
LinkConstraints:    21             (21)
 sub-OptiGraphs:     0              (0)
       OptiGraph: # elements (including subgraphs)
-------------------------------------------------------------------
      OptiNodes:    35             (35)
      OptiEdges:    17             (17)
LinkConstraints:    17             (17)
 sub-OptiGraphs:     0              (0)
       OptiGraph: # elements (including subgraphs)
-------------------------------------------------------------------
      OptiNodes:    43             (43)
      OptiEdges:    21             (21)
LinkConstraints:    21             (21)
 sub-OptiGraphs:     0              (0)
       OptiGraph: # elements (including subgraphs)
-------------------------------------------------------------------
      OptiNodes

In [11]:
# Optimize with Schwarz algorithm
# sub_optimizer = JuMP.optimizer_with_attributes(Ipopt.Optimizer, "tol"=>1e-12, "print_level" => 0, "hsllib"=>HSL_jll.libhsl_path, "linear_solver" => "ma57")
sub_optimizer = JuMP.optimizer_with_attributes(()->MadNLP.Optimizer(linear_solver=Ma57Solver))

# sub_optimizer = JuMP.optimizer_with_attributes(Gurobi.Optimizer)
# sub_optimizer = JuMP.optimizer_with_attributes(CPLEX.Optimizer)
# sub_optimizer = JuMP.optimizer_with_attributes(GLPK.Optimizer)

result = SchwarzOpt.optimize!(graph; subgraphs=expanded_subgraphs, sub_optimizer=sub_optimizer, max_iterations=100, mu=1.0)

Optimizing with user provided overlap
Initializing SchwarzOpt...
###########################################################
Optimizing with SchwarzOpt v0.2.0 using 1 threads
###########################################################

Number of variables: 199
Number of constraints: 299
Number of subproblems: 8
Overlap: 
Subproblem variables:   [43, 35, 43, 45, 43, 35, 43, 45]
Subproblem constraints: [43, 36, 43, 45, 43, 35, 43, 45]

This is MadNLP version v0.8.0, running with ma57

Number of nonzeros in constraint Jacobian............:      106
Number of nonzeros in Lagrangian Hessian.............:       43

Total number of variables............................:       43
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       21
Total number of inequality constraints...............:       43

SchwarzOpt.Optimizer

* Result configuration of SchwarzOpt.jl

In [13]:
## Query solution

# check the nodes of subgraphs.
partition = Plasmo.optinodes(subgraphs[2])
# print the decision variables corresponding to nodes in subgraph.
for i in 1:length(partition)
    if contains(partition[i].label, "state")
        println(partition[i].label, ": ", JuMP.value(partition[i][:x]))
    else
        println(partition[i].label, ": ", JuMP.value(partition[i][:u]))    
    end
end

state[1]: -1.5777218104420236e-30
state[2]: 0.13849006085859858
state[3]: 0.4832966245931278
state[4]: 0.5432223941548405
state[5]: 0.24844805450348292
state[6]: -9.999854469708408e-9
state[7]: -9.999931845346508e-9
state[8]: 0.017500020894656236
state[9]: 0.384871720584157
state[10]: 0.5598753794760263
state[11]: 0.3386148217126833
state[12]: -9.999495492248234e-9
state[13]: -9.999934458459462e-9
control[1]: -0.7029809239492979
control[2]: -0.5644908630911525
control[3]: -0.08119423849815449
control[4]: 0.46202815565657057
control[5]: 0.710476210159801
control[6]: 0.2794154981988485
control[7]: -0.639486567824201
control[8]: -0.6219865469338811
control[9]: -0.23711482634988737
control[10]: 0.3227605531260268
control[11]: 0.6613753748385247
control[12]: 0.536572917999996


In [14]:
## Query the results related to Schwarz algorithm.
# obj. value
result.objective_value

# current primal values that get communicated to subproblems
# result.x_vals

#  current dual values that get communicated to subproblems
# result.l_vals

# criterion of Schwarz algorithm: primal residual and dual residual
# result.err_pr
# result.err_du

33.838969013869544

In [15]:
## Query the time information to solving the problem using Schwarz algorithm.

# result.timers
result.timers.start_time
result.timers.initialize_time
result.timers.eval_objective_time
result.timers.eval_primal_feasibility_time
result.timers.eval_dual_feasibility_time
result.timers.update_subproblem_time
result.timers.solve_subproblem_time
result.timers.total_time

0.17474102973937988